this notebook is prepared by kevin zhu for SYDE 522.

references used:
- https://towardsdatascience.com/building-a-random-forest-classifier-c73a4cae6781
- Murphy 2022, Probabilistic Machine Learning: An Introduction
- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

### Import Data

Titanic dataset

https://www.kaggle.com/competitions/titanic/data

- survived:	Survived?	
    - 0 = No, 1 = Yes

- pclass:	Ticket class
	- 1 = 1st, 2 = 2nd, 3 = 3rd
- sex:	Sex	
    - male, female
- age:	Age in years	
- sibsp:	# of siblings / spouses aboard the Titanic	
- parch:	# of parents / children aboard the Titanic	
- fare:	Passenger fare	
- embark_town:	Port of Embarkation	
    - C = Cherbourg, Q = Queenstown, S = Southampton

In [2]:
# load data
titanic = sns.load_dataset('titanic')

Ycolumms = ['survived']
Xcolumns= ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embark_town']

titanic = titanic[Ycolumms + Xcolumns].dropna()

print('Dataset size:', titanic.shape)
print('Overview:')
titanic.head(10)

Dataset size: (712, 8)
Overview:


,survived,pclass,sex,age,sibsp,parch,fare,embark_town
0,0,3,male,22.0,1,0,7.2500,Southampton
1,1,1,female,38.0,1,0,71.2833,Cherbourg
2,1,3,female,26.0,0,0,7.9250,Southampton
3,1,1,female,35.0,1,0,53.1000,Southampton
4,0,3,male,35.0,0,0,8.0500,Southampton
6,0,1,male,54.0,0,0,51.8625,Southampton
7,0,3,male,2.0,3,1,21.0750,Southampton
8,1,3,female,27.0,0,2,11.1333,Southampton
9,1,2,female,14.0,1,0,30.0708,Cherbourg
10,1,3,female,4.0,1,1,16.7000,Southampton


In [3]:
print('Prob survival for female = {:.1f}%'.format(
    titanic['survived'][titanic['sex'] == 'female'].mean() * 100 
    ))

print('Prob survival for non first class female = {:.1f}%'.format(
    titanic['survived'][(titanic['sex'] == 'female') &
                        (titanic['pclass'] != 1)].mean() * 100 
    ))

Prob survival for female = 75.3%
Prob survival for non first class female = 65.3%


In [4]:
print('Prob survival for children under 15 = {:.1f}%'.format(
    titanic['survived'][titanic['age'] < 15].mean() * 100 
    ))

print('Prob survival for children under 15 that are alone = {:.1f}%'.format(
    titanic['survived'][(titanic['age'] < 15) &
                        (titanic['sibsp'] + titanic['parch'] == 0)].mean() * 100 
    ))

print('Number of children under 15 that are alone = {}'.format(
    len(titanic[(titanic['age'] < 15) &
                (titanic['sibsp'] + titanic['parch'] == 0)])
    ))

Prob survival for children under 15 = 57.7%
Prob survival for children under 15 that are alone = 50.0%
Number of children under 15 that are alone = 4


## 1. Decision trees 
(binary)

<div>
<img src="images/tree.png" width="700"/>
</div>

*values in image differ from the Titanic dataset loaded.

### How to determine split?

*We have to decide which feature to split on, and at what level. How?*

We will try every single feature and every single level, score each combination, and select the best..

```
for feature in all features:
    for level in feature levels:
        split data by level and evaluate how good the split is
select the best split
```
---

### How to evaluate split?

<u>Gini Index</u>

One commonly used metric is the Gini impurity score. 
Let $P_k$ be the percent of data that falls in class $k$, for $K$ total classes:
$$
\text{Gini} = 1- \sum_{k=1}^K P_k^2
$$

(we want a lower Gini index)

---

### Back to splitting algorithm:

After splitting the data into groups, we compute the Gini score for each group, then take the weighted sum of all groups as the Gini score for the split.

For example, if we split the below data by sex (female/male), we get a Gini score of 0.2.

<div>
<img src="images/gini.png" width="700"/>
</div>


In [5]:
from randomforest import find_gini

x = np.array([1,1,1,1,1,2,2,2])
y = np.array([1,1,1,1,0,0,0,0])

# split by female (x=1) and male (x=2):
group_female = np.where(x==1)[0]
group_male = np.where(x==2)[0]


split_gini = find_gini(group_female, group_male, y)

print('gini score for split {:.2f}'.format(split_gini))

gini score for split 0.20


### Next step?

Repeat for the left and right children of the previous split...

- when do we stop?



---

Back to Titanic..

In [6]:
# some data processing...

# convert categorical features to label encodings (or one hot encoding)
titanic = titanic.replace({'sex': {'male':0, 'female':1},
                           'embark_town': {'Cherbourg': 0, 'Queenstown':1, 'Southampton':2}})

X = titanic[Xcolumns] #predictors
y = titanic[Ycolumms].to_numpy() #response

# split data into train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Check root of decision tree (first split)

In [7]:
from randomforest import DecisionTree, RandomForest

# fit decision tree
tree = DecisionTree(X_train, y_train, min_leaf=5)
tree

n: 569; gini:[0.33325706]; split:0.0; var: sex

Check Gini score of root using our function:

In [8]:
# split training data by sex
group_female = np.where(X_train['sex']==1)[0]
group_male = np.where(X_train['sex']==0)[0]

# compute Gini score for split
split_gini = find_gini(group_female, group_male, y_train[:,0])

print('gini score for split {:.3f}'.format(split_gini))

gini score for split 0.333


Check children of root:

In [9]:
print('root_left', tree.lhs)
print('root_right', tree.rhs)

root_left n: 361; gini:[0.28214396]; split:1.0; var: pclass
root_right n: 208; gini:[0.25880759]; split:2.0; var: pclass


### True or False:

Decision trees
- are easy to interpret.
-  can easily handle mixed discrete and continuous inputs.
-  are insensitive to monotone transformations of the inputs, so there is no need to standardize the data.
- perform automatic variable selection.
- are relatively robust to outliers.
- can handle missing input features.

---

What are some potential issues with decision trees?

## 2. Random forest

Decision trees are a high variance estimator (i.e. unstable, predictions might vary a lot if the training data is perturbed).

A simple way to reduce variance is to average multiple models (<u>ensemble learning</u>).

In this case, we can fit multiple trees and average them.

---

How do we build "different" trees? Randomforests applies the 2 techniques below to build multiple trees in parallel.

#### 1) Bagging (bootstrap aggregating).

**What:** We fit different base models to different randomly sampled versions of the data to encourage the different models to make diverse predictions.

The datasets are sampled with replacement (bootstrap sampling).
If we sample with replacement $N$ times from our dataset of size $N$, the probability an observation is not in our bootstrapped dataset is $(1-\frac{1}{N})^N$.
\begin{align}
    \lim_{N\rightarrow \infty} (1-\frac{1}{N})^N = e^{-1} \approx \frac{1}{3}
\end{align}

The <u>OOB (out-of-bag)</u> samples are the $\approx\frac{1}{3}$ observations that were not selected to build a parcticular tree. We can use this as a "test set" to evalute the tree.

(*Note: Bagging does not always improve performance. In particular, it relies on the base models being unstable estimators, so that omitting some of the data significantly changes the resulting model fit.)

In [10]:
# implementation from randomforest.py

# using 10 trees
rf = RandomForest( X_train, y_train, n_trees=10, min_leaf = 1)

preds = rf.predict(X_test.to_numpy())

# prediction accuracy
print('prediction accuracy: {:.2f}'.format(metrics.accuracy_score(y_test, preds)))

prediction accuracy: 0.77


In [11]:
# sklearn implementation:
rf_sk = RandomForestClassifier(n_estimators=10, bootstrap=True, max_features=None, min_samples_leaf=1)

rf_sk.fit(X_train, y_train[:,0])

pred_scikit = rf_sk.predict(X_test)

print('(sklearn) prediction accuracy: {:.2f}'.format(metrics.accuracy_score(y_test, pred_scikit)))

(sklearn) prediction accuracy: 0.75


In [12]:
# without bootstrap sampling
rf_sk = RandomForestClassifier(n_estimators=10, bootstrap=False, max_features=None, min_samples_leaf=1)

rf_sk.fit(X_train, y_train[:,0])

pred_scikit = rf_sk.predict(X_test)

print('(sklearn no baggging) prediction accuracy: {:.2f}'.format(metrics.accuracy_score(y_test, pred_scikit)))

(sklearn no baggging) prediction accuracy: 0.71


#### 2) Limit the number of features to consider when looking for the best split.

For a dataset with $M$ total features, at each split, sample a subset of features (usually of size $\sqrt{M}$) to consider instead of all $M$ features.

In [16]:
rf_sk = RandomForestClassifier(n_estimators=10, bootstrap=True, max_features='sqrt', min_samples_leaf=1) 

rf_sk.fit(X_train, y_train[:,0])

pred_scikit = rf_sk.predict(X_test)

print('(sklearn rf) prediction accuracy: {:.2f}'.format(metrics.accuracy_score(y_test, pred_scikit)))

(sklearn rf) prediction accuracy: 0.78


In [26]:
rf_sk = RandomForestClassifier(n_estimators=50, bootstrap=True, max_features='sqrt', min_samples_leaf=1) 

rf_sk.fit(X_train, y_train[:,0])

pred_scikit = rf_sk.predict(X_test)

print('(sklearn rf, ntree = 50) prediction accuracy: {:.2f}'.format(metrics.accuracy_score(y_test, pred_scikit)))

(sklearn rf, ntree = 50) prediction accuracy: 0.80
